In [1]:
import numpy as np
import cv2
import os
import keras
from tqdm import tqdm_notebook
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input, Activation
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Conv2DTranspose, GaussianNoise, InputLayer
import matplotlib.pyplot as plt
from PIL import Image
from keras.models import load_model

Using TensorFlow backend.


In [2]:
IMAGE_SIZE = 0
TRAINING_IMAGES_PATH = None
RESULT_IMAGES_PATH = None
EPOCHS = 0
BATCH_SIZE = 0

def read_settings():
    global IMAGE_SIZE
    global TRAINING_IMAGES_PATH
    global RESULT_IMAGES_PATH
    global EPOCHS
    global BATCH_SIZE
    f = open(os.path.abspath("settings.txt"), "r")
    line = f.readline()
    TRAINING_IMAGES_PATH = line.split('=')[1].replace('\n','')
    line = f.readline()
    RESULT_IMAGES_PATH = line.split('=')[1].replace('\n','')
    line = f.readline()
    IMAGE_SIZE = int(line.split('=')[1].replace('\n',''))
    line = f.readline()
    EPOCHS = int(line.split('=')[1].replace('\n',''))
    line = f.readline()
    BATCH_SIZE = int(line.split('=')[1].replace('\n', ''))

read_settings()

In [3]:
TRAINING_SAMPLES = len(os.listdir(TRAINING_IMAGES_PATH))
TRAINING_DATA = np.empty([TRAINING_SAMPLES, IMAGE_SIZE, IMAGE_SIZE, 3])

def read_images():
    iterator = 0
    for file in tqdm_notebook(os.listdir(TRAINING_IMAGES_PATH)):
        image = cv2.imread(TRAINING_IMAGES_PATH + file)
        img = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        im_array = np.asarray(img) / 255
        TRAINING_DATA[iterator] = im_array
        iterator += 1
        
read_images()

In [4]:
def create_generator():
    epsilon = 0.00001 # Small float added to variance to avoid dividing by zero in the BatchNorm layers.
    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(4*4*512, activation='linear', input_shape=(100,)))
    model.add(Reshape((4, 4, 512)))

    model.add(Conv2DTranspose(256, kernel_size=[5,5], strides=[2,2], padding="same",
                              kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
    model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(256, kernel_size=[5,5], strides=[2,2], padding="same",
                              kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
    model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(128, kernel_size=[5,5], strides=[2,2], padding="same",
                              kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
    model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
    model.add(Activation('relu'))
    
    model.add(Conv2DTranspose(64, kernel_size=[5,5], strides=[2,2], padding="same",
                              kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
    model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(3, kernel_size=[5,5], strides=[1,1], padding="same",
                              kernel_initializer= keras.initializers.TruncatedNormal(stddev=0.02)))
    # Standard activation for the generator of a GAN
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(100,))
    img = model(noise)

    return Model(noise, img)

generator = create_generator()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8192)              827392    
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 256)         3277056   
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 256)         1024      
_________________________________________________________________
activation_1 (Activation)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 16, 16, 256)       1638656   
_________________________________________________________________
batc

In [5]:
def build_discriminator():

    model = Sequential()

    model = Sequential()
    model.add(InputLayer(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3)))
    model.add(GaussianNoise(0.2))
    model.add(Conv2D(32, (3, 3), strides=[2,2],padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dropout(.3))
    model.add(Conv2D(64, (3, 3),strides=[2,2], kernel_initializer='he_uniform', padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dropout(.3))
    model.add(Conv2D(128, (3, 3),strides=[2,2], kernel_initializer='he_uniform', padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dropout(.3))
    model.add(Conv2D(256, (3, 3),strides=[2,2], kernel_initializer='he_uniform', padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dropout(.3))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
        
    model.summary()
        
    img = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    validity = model(img)

    return Model(img, validity)

discriminator = build_discriminator()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise_1 (GaussianNo (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
___________________________

In [6]:
def get_gan():
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.summary()
    
    return model

gan = get_gan()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 64, 64, 3)         6774915   
_________________________________________________________________
model_2 (Model)              (None, 1)                 394433    
Total params: 7,169,348
Trainable params: 7,166,980
Non-trainable params: 2,368
_________________________________________________________________


In [7]:
optimizer = Adam(0.0002,0.5)
discriminator.compile(loss='binary_crossentropy', 
                                   optimizer=optimizer,
                                   metrics=['accuracy'])
discriminator.trainable = False
generator.compile(loss='binary_crossentropy', optimizer=optimizer)
gan.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [8]:
from PIL import Image

def save_images(epoch):
        noise = np.random.normal(0, 1, [100, 100])
        image_array = np.empty([25, IMAGE_SIZE, IMAGE_SIZE,3])
        gen_imgs = generator.predict(noise)
        image_count = 0
        for i in range(5):
                image_array[i] = gen_imgs[i]
                im = np.asarray(image_array[i] * 255)
                cv2.imwrite(RESULT_IMAGES_PATH + "image_%d.png" % (epoch + i) ,im)

In [9]:
discriminator_loss_acc = []
gan_loss_acc = []

def train_gan():
    
    for i in range(EPOCHS):
        # Initialize loss and acc for the discriminator / generator
        d_loss_acc = [0, 0]
        g_loss_acc = [0, 0]
        # Create the nose input of shape batch_size, 100
        noise = np.random.normal(0, 1, [BATCH_SIZE, 100])
        # Create fake images using the generator
        fake_images = generator.predict(noise)
        # Create the coresponding output for fake images 
        fake_output = np.full((BATCH_SIZE, 1), 0)
        # Index of 64 real real images
        indexes = np.random.randint(0, len(os.listdir(TRAINING_IMAGES_PATH)), 64)
        # Get a batch of real images
        real_images = TRAINING_DATA[indexes]
        # Output for the real image is 0.9 (https://machinelearningmastery.com/how-to-train-stable-generative-adversarial-networks/)
        real_output = np.full((BATCH_SIZE, 1), 0.9)
        # Train discrimnator on the real and fake images
        d_loss_acc = np.add(d_loss_acc, discriminator.train_on_batch(fake_images, fake_output))
        d_loss_acc = np.add(d_loss_acc, discriminator.train_on_batch(real_images, real_output))
        d_loss_acc = d_loss_acc * 0.5
        # Input data for training the generator
        noise = np.random.normal(0,1, [BATCH_SIZE, 100])
        g_loss_acc = np.add(g_loss_acc, gan.train_on_batch(noise,np.full((BATCH_SIZE, 1), 1)))
        if i % 1000 == 0 and i != 0:
            discriminator.save(RESULT_IMAGES_PATH + 'traffic_discriminator.h5')  # creates a HDF5 file 'my_model.h5'
            generator.save(RESULT_IMAGES_PATH + 'traffic_generator.h5')
            print(d_loss_acc)
            print(g_loss_acc)
            discriminator_loss_acc.append(d_loss_acc)
            gan_loss_acc.append(g_loss_acc)
            save_images(i)

In [10]:
train_gan()

C:\Users\Cordu\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Instructions for updating:
Use tf.cast instead.
[0.96640664 0.2421875 ]
[1.14441466 0.15625   ]
[0.58241394 0.453125  ]
[0.80984199 0.421875  ]
[0.7331959 0.34375  ]
[0.81671774 0.390625  ]
[0.69146562 0.421875  ]
[0.92894346 0.171875  ]
[0.6967504 0.4140625]
[0.9367097 0.171875 ]
[0.62292331 0.421875  ]
[0.94000679 0.234375  ]
[0.68659562 0.3515625 ]
[0.98249054 0.21875   ]
[0.61086392 0.40625   ]
[0.90135455 0.3125    ]
[0.71450886 0.3359375 ]
[0.9439339 0.1875   ]
[0.64077556 0.3828125 ]
[0.94196558 0.21875   ]
[0.64401525 0.375     ]
[1.05308914 0.171875  ]
[0.68103746 0.3046875 ]
[0.95475292 0.25      ]
[0.61684884 0.4140625 ]
[1.13159728 0.140625  ]
[0.53857481 0.4453125 ]
[1.2212323 0.15625  ]
[0.67967033 0.3828125 ]
[1.17829633 0.125     ]
[0.61900482 0.3671875 ]
[1.24732816 0.15625   ]
[0.54618534 0.3671875 ]
[1.24631405 0.125     ]
[0.60662818 0.375     ]
[1.07266212 0.21875   ]
[0.53533143 0.3984375 ]
[1.49974072 0.09375   ]
[0.49957418 0.375     ]
[1.4510082 0.15625  ]
[0.5